In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import sqlite3

In [2]:
news_dict = {'title':[],'href':[],'time':[],'acticle':[]}
for h in tqdm(range(1,11)):
    r = requests.get('http://edu.sina.com.cn/other/roll.d.html?cat=80459&page={page}&page_size=30'.format(page = h))
    r.encoding = 'utf-8'
    soup = BeautifulSoup(r.text,'html.parser')
    new =soup.find_all('li')
    for i in  range(len(new)):
        href = new[i]('a')[0]['href']
        data = new[i]('span')[0].text.strip('()')
        title = new[i].text
        news_dict['title'].append(title)
        news_dict['href'].append(href)
        news_dict['time'].append(data)
for i in tqdm(range(0,300)):
    try:
        res = requests.get(news_dict['href'][i])
        res.encoding = 'utf-8'
        soup = BeautifulSoup(res.text,'html.parser')
        new = soup.find('div','article')
        st1=new.text.replace(u'\n\u3000\u3000', u' ').replace(u'\n\n\n\n',u'').replace(u'\n\n',u'')
    except:
        print('地址为{}新闻抓取失败'.format(news_dict['href'][i]))
        st1 = '没有获取数据'
        continue
    
    news_dict['acticle'].append(st1)

100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [01:03<00:00,  4.75it/s]


In [3]:
df = pd.DataFrame.from_dict(news_dict, orient='index')

In [4]:
df.transpose().head(122)

,title,href,time,acticle
0,教育部：审慎研究制定今年高考实施方案(2020年02月29日 13:04),https://edu.sina.com.cn/gaokao/2020-02-29/doc-...,2020年02月29日 13:04,\t\t\t\t\twindow.sina_survey_config = {\n \...
1,今年高考会不会推迟？教育部回应(2020年02月29日 13:02),https://edu.sina.com.cn/gaokao/2020-02-29/doc-...,2020年02月29日 13:02,\t\t\t\t\twindow.sina_survey_config = {\n \...
2,福建：高三质量检测3月28-29日按原计划进行(2020年02月28日 21:02),https://edu.sina.com.cn/gaokao/2020-02-28/doc-...,2020年02月28日 21:02,\t\t\t\t\twindow.sina_survey_config = {\n \...
3,青海：疫情防控期间教育招生考试有关工作安排(2020年02月28日 20:57),https://edu.sina.com.cn/gaokao/2020-02-28/doc-...,2020年02月28日 20:57,\t\t\t\t\twindow.sina_survey_config = {\n \...
4,广东：华侨港澳台全国联招3月1日起网上报名(2020年02月28日 20:54),https://edu.sina.com.cn/gaokao/2020-02-28/doc-...,2020年02月28日 20:54,\t\t\t\t\twindow.sina_survey_config = {\n \...
...,...,...,...,...
117,吉林：2020高招舞蹈类专业统一考试合格分数线(2020年02月10日 18:36),http://edu.sina.com.cn/gaokao/2020-02-10/doc-i...,2020年02月10日 18:36,\t\t\t\t\twindow.sina_survey_config = {\n \...
118,重庆市：2020年普通高考报名须知(2020年02月10日 18:25),http://edu.sina.com.cn/gaokao/2020-02-10/doc-i...,2020年02月10日 18:25,\t\t\t\t\twindow.sina_survey_config = {\n \...
119,重庆：高招艺术类专业考试成绩最低控制分数线(2020年02月10日 18:20),http://edu.sina.com.cn/gaokao/2020-02-10/doc-i...,2020年02月10日 18:20,\t\t\t\t\twindow.sina_survey_config = {\n \...
120,2020年上海普通高中学业水平合格性考试顺利结束(2020年02月10日 18:17),http://edu.sina.com.cn/gaokao/2020-02-10/doc-i...,2020年02月10日 18:17,\t\t\t\t\twindow.sina_survey_config = {\n \...


In [5]:
df.to_excel('高考.xlsx')

In [8]:
cx = sqlite3.connect(r'D:\Python\获取网络信息用于文本分析\data.db')
pd.io.sql.to_sql(df,'gaokao',con = cx,if_exists='append')

In [9]:
df2 = pd.read_sql_query('select * from gaokao',cx)

In [10]:
df2.transpose().head(100)

,0,1,2,3
index,title,href,time,acticle
0,教育部：审慎研究制定今年高考实施方案(2020年02月29日 13:04),https://edu.sina.com.cn/gaokao/2020-02-29/doc-...,2020年02月29日 13:04,\t\t\t\t\twindow.sina_survey_config = {\n \...
1,今年高考会不会推迟？教育部回应(2020年02月29日 13:02),https://edu.sina.com.cn/gaokao/2020-02-29/doc-...,2020年02月29日 13:02,\t\t\t\t\twindow.sina_survey_config = {\n \...
2,福建：高三质量检测3月28-29日按原计划进行(2020年02月28日 21:02),https://edu.sina.com.cn/gaokao/2020-02-28/doc-...,2020年02月28日 21:02,\t\t\t\t\twindow.sina_survey_config = {\n \...
3,青海：疫情防控期间教育招生考试有关工作安排(2020年02月28日 20:57),https://edu.sina.com.cn/gaokao/2020-02-28/doc-...,2020年02月28日 20:57,\t\t\t\t\twindow.sina_survey_config = {\n \...
...,...,...,...,...
94,甘肃：推迟举行2020年高招艺术类专业校考(2020年02月12日 15:20),http://edu.sina.com.cn/gaokao/2020-02-12/doc-i...,2020年02月12日 15:20,\t\t\t\t\twindow.sina_survey_config = {\n \...
95,上海：教育考试院暂停现场接待来访和办理成绩证明(2020年02月12日 15:17),http://edu.sina.com.cn/gaokao/2020-02-12/doc-i...,2020年02月12日 15:17,\t\t\t\t\twindow.sina_survey_config = {\n \...
96,广西：普通高考加分调整方案政策解读(2020年02月12日 13:07),http://edu.sina.com.cn/gaokao/2020-02-12/doc-i...,2020年02月12日 13:07,\t\t\t\t\twindow.sina_survey_config = {\n \...
97,广西：普通高考加分调整方案咨询电话(2020年02月12日 13:05),http://edu.sina.com.cn/gaokao/2020-02-12/doc-i...,2020年02月12日 13:05,\t\t\t\t\twindow.sina_survey_config = {\n \...
